In [ ]:
!nvidia-smi

Fri Jan  9 10:41:55 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/KoelLabs/ML.git

fatal: destination path 'ML' already exists and is not an empty directory.


In [ ]:
%cd /content/ML/notebooks/Finetuning

/content/ML/notebooks/Finetuning


In [ ]:
!pip install ffmpeg sounddevice ipapy torchcodec panphon jiwer

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.0 MB/s eta 0:00:00


In [ ]:
import sys
import types

fake_sd = types.ModuleType("sounddevice")

def _not_supported(*args, **kwargs):
    raise RuntimeError("sounddevice is not supported on Colab")

fake_sd.play = _not_supported
fake_sd.rec = _not_supported
fake_sd.wait = _not_supported
fake_sd.stop = _not_supported

sys.modules["sounddevice"] = fake_sd


In [ ]:
import os
import sys
sys.path.append(os.path.join("..", ".."))
from scripts.eval.evaluate import evaluate
from scripts.core.audio import audio_file_to_array
from scripts.ipa_transcription.wav2vec2 import transcribe_batch, clear_cache
from scripts.data_loaders.common import show_hf_sample
from scripts.ipa_transcription.wav2vec2_train import (
    load_datasets, inspect_samples, combine_datasets, plot_dataset_distributions, get_or_create_processed_dataset,
    identify_dataset_vocab, identify_model_vocab, align_dataset_with_model_vocab, save_model_with_updated_vocab,
    prepare_model_and_processor, DataCollatorCTCWithPadding,
)

import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor

import torch
from transformers import Trainer, TrainingArguments

from IPython.display import clear_output
clear_output()

In [ ]:
PROJECT_NAME = "wav2vec2_ipa_condensed"
WANDB_TEAM="taikhoanboss1-daijiworld-com"
PRE_TRAINED_ID = "facebook/wav2vec2-xlsr-53-phon-cv-ft"
CONDENSED_MODEL_DIR = os.path.join("..", "..", "models", "facebook53_condensed")
PROCESSED_DIR = os.path.join("..", "..", ".data", "processed_ipa")

NUM_PROC = max(1, os.cpu_count() - 1)  # type: ignore
RANDOM_SEED = 42  # KEEP CONST

In [ ]:
import os
os.environ["WANDB_PROJECT"] = PROJECT_NAME
import wandb
wandb.login(key="wandb_key_here")  # replace with your actual WandB key
from huggingface_hub import login
login(token="hunggingdface_hub_token_here")  # replace with your actual Hugging Face token
import logging
logging.getLogger("wandb").setLevel(logging.ERROR)
import warnings
warnings.filterwarnings(action="ignore", category=UserWarning)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: taikhoanboss1 (taikhoanboss1-daijiworld-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
datasets = load_datasets()
clear_output()

In [ ]:
combined_ds = combine_datasets(datasets, seed=RANDOM_SEED)

In [ ]:
dataset_vocab, symbol_uses = identify_dataset_vocab(combined_ds)
print(dataset_vocab)
# Determine the set of symbols supported by the pretrained model
model_vocab, special_tokens = identify_model_vocab(PRE_TRAINED_ID)
print(model_vocab)
print(special_tokens)

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

{'e', 'ɛ', 'ɔɪ', 'ʒ', 'o', 'oʊ', 'k', 'ɡ', 'z', 'ʌ', 'ð', 'n', 'w', 'ə', 'tʃ', 'ɹ', 'd', 'f', 'ɑ', 's', 'aʊ', 'θ', 'eɪ', 'ɔ', 'u', 'ŋ', 'aɪ', 'p', 't', 'dʒ', 'i', 'm', 'j', 'ʊ', 'ɪ', 'l', 'h', 'ʃ', 'v', 'æ', 'ɝ', 'a', 'b'}
{'<s>': 1, '<pad>': 0, '</s>': 2, '<unk>': 3, 'n': 4, 't': 5, 's': 6, 'a': 7, 'ɪ': 8, 'l': 9, 'ə': 10, 'd': 11, 'ɛ': 12, 'e': 13, 'k': 14, 'i': 15, 'm': 16, 'o': 17, 'p': 18, 'z': 19, 'ʁ': 20, 'b': 21, 'v': 22, 'f': 23, 'j': 24, 'r': 25, 'ɻ': 26, 'u': 27, 'w': 28, 'ɔ': 29, 'æ': 30, 'ʊ': 31, 'ɒ': 32, 'iː': 33, 'ɾ': 34, 'ʃ': 35, 'h': 36, 'θ': 37, 'eɪ': 38, 'ŋ': 39, 'y': 40, 'ð': 41, 'R': 42, 'ɡ': 43, 'ɜ': 44, 'g': 45, 'uː': 46, 'ɔː': 47, 'aɪ': 48, 'ʌ': 49, 'x': 50, 'aː': 51, 'əʊ': 52, 'ɑː': 53, 'ɜː': 54, 'n̩': 55, 'ɑ': 56, 'eː': 57, 'ɲ': 58, 'ʒ': 59, 'dʒ': 60, 'l̩': 61, 'ɑ̃': 62, 'ai': 63, 'ts': 64, 'β': 65, 'œ': 66, 'oː': 67, 'aʊ': 68, 'ɔ̃': 69, 'ʏ': 70, 'ɪə': 71, 'ɛə': 72, 'ɥ': 73, 'ʂ': 74, 'ɕ': 75, 'au': 76, 'ɨ': 77, 'c': 78, 'ɯ': 79, 'ɫ': 80, 'ʔ': 81, '’': 82, 'ɛ̃'

In [ ]:
# Reconcile model vocab with dataset vocab to see which weights we can transfer
matched_tokens, partial_matched_tokens, unmatched = align_dataset_with_model_vocab(dataset_vocab, model_vocab, special_tokens)

print('Matched', matched_tokens)
print('Partial Matched', partial_matched_tokens)
print('Unmatched', unmatched)
# Save updated tokenizer with the reconciled vocab
vocab_dict = save_model_with_updated_vocab(CONDENSED_MODEL_DIR, PRE_TRAINED_ID, special_tokens, matched_tokens, partial_matched_tokens, seed=RANDOM_SEED)
print(vocab_dict)# Save updated tokenizer with the reconciled vocab
vocab_dict = save_model_with_updated_vocab(CONDENSED_MODEL_DIR, PRE_TRAINED_ID, special_tokens, matched_tokens, partial_matched_tokens, seed=RANDOM_SEED)
print(vocab_dict)
# sanity check
model, processor = prepare_model_and_processor(CONDENSED_MODEL_DIR)
wav = audio_file_to_array(os.path.join("..", "..", "data", "ExamplesWithComments", "TIMIT_sample_0.wav"))
print(transcribe_batch([(None, wav)], model, processor)[0]) # type: ignore

# should look something like:
# prev = "aɪɻaɪtæktəstʌmptəweɪɪdɪɒtɪklɪ" (facebook)
# curr = "aɪraɪtæktəstʌmptəweɪɪdɪɒtɪklɪ" (new)
del wav, model, processor
clear_cache()

Matched {'e': 13, 'ɛ': 12, 'ɔɪ': 92, 'ʒ': 59, 'o': 17, 'k': 14, 'ɡ': 43, 'z': 19, 'ʌ': 49, 'ð': 41, 'n': 4, 'w': 28, 'ə': 10, 'tʃ': 111, 'ɹ': 105, 'd': 11, 'f': 23, 'ɑ': 56, 's': 6, 'aʊ': 68, 'θ': 37, 'eɪ': 38, 'ɔ': 29, 'u': 27, 'ŋ': 39, 'aɪ': 48, 'p': 18, 't': 5, 'dʒ': 60, 'i': 15, 'm': 16, 'j': 24, 'ʊ': 31, 'ɪ': 8, 'l': 9, 'h': 36, 'ʃ': 35, 'v': 22, 'æ': 30, 'ɝ': 108, 'a': 7, 'b': 21}
Partial Matched {'oʊ': [17, 31]}
Unmatched set()
Model weight transfer complete
   old head rows : 272
   new head rows : 47
🚀  Condensed checkpoint saved to ../../models/facebook53_condensed
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, 'n': 4, 't': 5, 's': 6, 'a': 7, 'ɪ': 8, 'l': 9, 'ə': 10, 'd': 11, 'ɛ': 12, 'e': 13, 'k': 14, 'i': 15, 'm': 16, 'o': 17, 'oʊ': 18, 'p': 19, 'z': 20, 'b': 21, 'v': 22, 'f': 23, 'j': 24, 'u': 25, 'w': 26, 'ɔ': 27, 'æ': 28, 'ʊ': 29, 'ʃ': 30, 'h': 31, 'θ': 32, 'eɪ': 33, 'ŋ': 34, 'ð': 35, 'ɡ': 36, 'aɪ': 37, 'ʌ': 38, 'ɑ': 39, 'ʒ': 40, 'dʒ': 41, 'aʊ': 42, 'ɔɪ': 43, 'ɹ': 44, 'ɝ'

In [ ]:
_, processor = prepare_model_and_processor(CONDENSED_MODEL_DIR)
processed_ds = get_or_create_processed_dataset(PROCESSED_DIR, combined_ds, processor, NUM_PROC, RANDOM_SEED)

Filter:   0%|          | 0/3599 [00:00<?, ? examples/s]

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/3599 [00:00<?, ? examples/s]

In [ ]:
split_dataset = processed_ds.train_test_split(test_size=0.2, seed=RANDOM_SEED) # type: ignore
train_ds = split_dataset['train']
val_ds = split_dataset['test']

In [ ]:
# Inspect processed data to make sure it is correct
batch = next(iter(train_ds))
labels = batch["labels"]  # type: ignore
print("Source: ", batch['ipa'])  # type: ignore
print("Decoded: ", processor.batch_decode([labels], group_tokens=False)[0])
print("Label min:", min(labels))
print("Label max:", max(labels))
print("Label unique:", set(labels))
del processor
clear_cache()

Source:  bʌˈdɛˈɹwʌˈzsʌˈmθɪŋiˈvɪmmɔˈɹstɑˈɹtlɪŋdæˈndɪˈsɹizɛˈmblʌns
Decoded:  bʌdɛɹwʌzsʌmθɪŋivɪmmɔɹstɑɹtlɪŋdændɪsɹizɛmblʌns
Label min: 4
Label max: 44
Label unique: {4, 5, 6, 8, 9, 11, 12, 15, 16, 20, 21, 22, 26, 27, 28, 32, 34, 38, 39, 44}


In [ ]:
SWEEP_TRIALS = 10
SWEEP_CONFIG = {
    "name": "freeze_feature_encoder_run2",
    "method": "random",
    "metric": {"name": "eval_fer", "goal": "minimize"},
    "parameters": {
        # optimiser
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 3e-5,
            "max": 5e-4,
        },
        "weight_decay": {"values": [0.0, 0.01, 0.05]},
        # training regime
        # effective batch size is train_batch_size * gradient_accumulation_steps for gradient calculation
        # but train_batch_size is what's actually kept in VRAM simultaneously
        # tweak appropriately for the amount of VRAM you have and the size of the data samples
        # for this data:
        #   - train_batch_size=8 is max with 40 GB VRAM with freeze_feature_extractor=False
        #   - train_batch_size=6 is max with 40 GB VRAM with freeze_feature_extractor=True
        "train_batch_size": {"values": [8]},
        "gradient_accumulation_steps": {"values": [2, 4]},
        "num_train_epochs": {"values": [5, 7, 10]},
        "warmup_steps": {"values": [200, 500, 800]},
        # SpecAugment (time masking prob)
        "mask_time_prob": {
            "distribution": "uniform",
            "min": 0.05,
            "max": 0.15,
        },
        "mask_feature_prob": {
            "distribution": "uniform",
            "min": 0.0,
            "max": 0.2,
        },
        # which weights to tune
        "freeze_feature_extractor": {"values": [True]},
    },
}

In [ ]:
def train_model(config=None):
    """
    Launches one training run with hyper-params from wandb.config.
    """
    run = wandb.init(config=config, project=PROJECT_NAME, entity=WANDB_TEAM, resume="allow")
    try:
        cfg = wandb.config

        # 1) model / processor ------------------------------------------------------
        model, processor = prepare_model_and_processor(
            CONDENSED_MODEL_DIR,
            freeze_feature_extractor=cfg.freeze_feature_extractor,
        )

        collator = DataCollatorCTCWithPadding(
            processor,
            padding="longest",
            pad_to_multiple_of=8,  # helps Tensor Cores
        )

        # 3) evaluation metric (PER) -----------------------------------------------
        def preprocess_logits_for_metrics(logits, labels):
            return torch.argmax(logits, dim=-1)

        def compute_metrics(pred):
            pred_ids = pred.predictions
            pred_text = processor.batch_decode(pred_ids)

            # replace -100 with pad_token_id before decoding
            label_ids = np.where(
                pred.label_ids != -100, pred.label_ids, processor.tokenizer.pad_token_id # type: ignore
            )
            label_text = processor.batch_decode(label_ids, group_tokens=False)

            with ProcessPoolExecutor(max_workers=NUM_PROC) as executor:
                scores = list(executor.map(evaluate, label_text, pred_text))
            return {
                "per": np.mean([s[0] for s in scores]),
                "fer": np.mean([s[1] for s in scores]),
            }

        # 4) HF Trainer -------------------------------------------------------------
        run_dir = os.path.join("runs", run.id)
        args = TrainingArguments(
            output_dir=run_dir,
            run_name=f"experiment_{run.id}",

            learning_rate=cfg.learning_rate,
            warmup_steps=cfg.warmup_steps,
            weight_decay=cfg.weight_decay,

            per_device_train_batch_size=cfg.train_batch_size,
            per_device_eval_batch_size=cfg.train_batch_size,
            gradient_accumulation_steps=cfg.gradient_accumulation_steps,

            fp16=torch.cuda.is_available(),

            num_train_epochs=cfg.num_train_epochs,
            eval_accumulation_steps=10,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_steps=50,
            save_total_limit=2,

            dataloader_num_workers=NUM_PROC,
            gradient_checkpointing=True,
        )

        # propagate SpecAugment param
        model.config.apply_spec_augment = True
        model.config.mask_time_prob = cfg.mask_time_prob
        model.config.mask_feature_prob = cfg.mask_feature_prob

        # divide losses by target lengths for each sample, then average by batch
        model.config.ctc_loss_reduction='mean'

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            data_collator=collator,
            compute_metrics=compute_metrics,
            preprocess_logits_for_metrics=preprocess_logits_for_metrics,
        )

        try:
            trainer.train(resume_from_checkpoint=True)
        except ValueError:
            trainer.train()
        eval_metrics = trainer.evaluate()
        wandb.log(eval_metrics)
        run.finish()
    finally:
        if 'trainer' in locals() or 'trainer' in globals():
            del trainer
        if 'model' in locals() or 'model' in globals():
            del model
        if 'processor' in locals() or 'processor' in globals():
            del processor
        clear_cache()

In [ ]:
# create the sweep by spawning an agent that runs `SWEEP_TRIALS` jobs.
# pro tip: make sure you have enough disk space with `df -h` (check individual folders with `du -sh <folder_path>`)
sweep_id = wandb.sweep(SWEEP_CONFIG, project=PROJECT_NAME, entity=WANDB_TEAM)
wandb.agent(sweep_id, function=train_model, count=SWEEP_TRIALS, project=PROJECT_NAME, entity=WANDB_TEAM)
print("🚀 sweep launched lets goooo:", sweep_id)

Create sweep with ID: lrkkim54
Sweep URL: https://wandb.ai/taikhoanboss1-daijiworld-com/wav2vec2_ipa_condensed/sweeps/lrkkim54


wandb: Agent Starting Run: t2zuks23 with config:
wandb: 	freeze_feature_extractor: True
wandb: 	gradient_accumulation_steps: 4
wandb: 	learning_rate: 0.00040351034028279493
wandb: 	mask_feature_prob: 0.07413776165531494
wandb: 	mask_time_prob: 0.13772397761400113
wandb: 	num_train_epochs: 5
wandb: 	train_batch_size: 8
wandb: 	warmup_steps: 200
wandb: 	weight_decay: 0.01


wandb: WARNING Config item 'mask_time_prob' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'mask_feature_prob' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Per,Fer
1,1.732500,0.624617,0.331984,0.036914
2,0.927800,0.574771,0.322194,0.037893


In [ ]:
# best settings from hypersweeps, but add more epochs
BEST_CONFIG = {
    "learning_rate": 0.000111,
    "weight_decay": 0.01,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 4,
    "num_train_epochs": 20,
    "warmup_steps": 800,
    "mask_time_prob": 0.06027,
    "mask_feature_prob": 0.1814,
    "freeze_feature_extractor": True,
}
RUN_NAME = "xlsr-english-l2"

In [ ]:
# model processor and collator
model, processor = prepare_model_and_processor(
    CONDENSED_MODEL_DIR,
    freeze_feature_extractor=BEST_CONFIG["freeze_feature_extractor"],
)

collator = DataCollatorCTCWithPadding(
    processor,
    padding="longest",
    pad_to_multiple_of=8,
)

In [ ]:
# apply settings
run_dir = os.path.join("..", "..", "models", RUN_NAME)
args = TrainingArguments(
    output_dir=run_dir,
    run_name=RUN_NAME,

    learning_rate=BEST_CONFIG["learning_rate"],
    warmup_steps=BEST_CONFIG["warmup_steps"],
    weight_decay=BEST_CONFIG["weight_decay"],

    per_device_train_batch_size=BEST_CONFIG["train_batch_size"],
    gradient_accumulation_steps=BEST_CONFIG["gradient_accumulation_steps"],

    fp16=torch.cuda.is_available(),

    num_train_epochs=BEST_CONFIG["num_train_epochs"],
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=BEST_CONFIG["num_train_epochs"],

    # disable eval
    do_eval=False,
    eval_strategy="no",

    dataloader_num_workers=NUM_PROC,
    gradient_checkpointing=True,
)

# propagate SpecAugment param
model.config.apply_spec_augment = True
model.config.mask_time_prob = BEST_CONFIG["mask_time_prob"]
model.config.mask_feature_prob = BEST_CONFIG["mask_feature_prob"]
    
# divide losses by target lengths for each sample, then average by batch
model.config.ctc_loss_reduction = 'mean'

In [ ]:
# train on full dataset
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=processed_ds, # type: ignore
    data_collator=collator,
)

try:
    trainer.train(resume_from_checkpoint=True)
except ValueError:
    trainer.train()

In [ ]:
if 'trainer' in locals() or 'trainer' in globals():
    del trainer
if 'model' in locals() or 'model' in globals():
    del model
if 'processor' in locals() or 'processor' in globals():
    del processor
clear_cache()

In [ ]:
# save model to huggingface
# make sure you're logged in, e.g., with `huggingface-cli login`: https://huggingface.co/docs/huggingface_hub/en/guides/cli#huggingface-cli-login
from huggingface_hub import HfApi
from transformers import AutoProcessor, AutoModelForCTC

best_model_dir = os.path.join(run_dir, "checkpoint-10980")
processor = AutoProcessor.from_pretrained(CONDENSED_MODEL_DIR)
model = AutoModelForCTC.from_pretrained(best_model_dir)

essentials_dir = os.path.join("..", "..", "models", RUN_NAME, "final")
model.save_pretrained(essentials_dir)
processor.save_pretrained(essentials_dir)

REPO_ID = 'Lam-Hung/xlsr-english-l2'
api = HfApi()
api.create_repo(REPO_ID, private=True)
api.upload_folder(
    repo_id=REPO_ID,
    folder_path=essentials_dir,
)